In [ ]:
from pcdsdevices.mirror import OffsetMirror
from pcdsdevices.mirror import PointingMirror
import datetime
from functions import *

import logging
# create logger
level = logging.INFO
log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
handlers = [logging.FileHandler('HOMS_test.log'), logging.StreamHandler()]
logging.basicConfig(level = level, format = log_format, handlers = handlers)
logger = logging.getLogger(__name__)
logger.info("\nTest initiated at: %s\n" %datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

In [ ]:
if get_user_confirmation() == False:
    import sys; sys.exit("aa! exit!!")

In [ ]:
fee_m1 = OffsetMirror('FEE1:M1H', name='fee_m1', prefix_xy='STEP:M1H', xgantry_prefix='STEP:FEE1:611:MOTR')
fee_m2 = OffsetMirror('FEE1:M2H', name='fee_m2', prefix_xy='STEP:M2H', xgantry_prefix='STEP:FEE1:861:MOTR')
xrt_m1 = PointingMirror('XRT:M1H', name='xrt_m1')
xrt_m2 = PointingMirror('XRT:M2H', name='xrt_m2')
xrt_m3 = OffsetMirror('XRT:M3H', name='xrt_m3')

In [ ]:
while True:
    try:
        which_mirror = int(input("which mirror do you want to use?\n press:\n 1 -> fee_m1; 2 -> fee_m2; 3 -> xrt_m1; 4 -> xrt_m2; 5  -> xrt_m3."))
    except ValueError:
        print ("sorry, I didn't understand that")
        continue
    if not which_mirror in range (1,6):
        print("sorry, it's not one of possible inputs")
        continue
    else:
        break
        
which_mirror_dict = {1: fee_m1, 2:fee_m2, 3:xrt_m1, 4:xrt_m2, 5:xrt_m3}
test_object = which_mirror_dict[which_mirror]
logger.info("\tusing %s mirror" %test_object.name)
# general timeout
test_object.xgantry.timeout = 300
test_object.yganry.timeout = 300
test_object.pitch.timetou = 300

In [ ]:
# FOR ALL AXES
logger.info("\t%.4f recording original position...\n" %test_object.xgantry.readback.timestamp)
initial_mirror_angle = test_object.pitch()
mirror_angle_unit = test_object.pitch.egu
initial_xgantry_value = test_object.xgantry()
initial_ygantry_value = test_object.ygantry()
gantry_value_unit = test_object.xgantry.egu
logger.info("initial mirror angle is %.4f %s" %(initial_mirror_angle, test_object.pitch.egu))
logger.info("initial xgantry value is %.4f %s" %(initial_xgantry_value, test_object.xgantry.egu))
logger.info("initial ygantry value is %.4f %s" %(initial_ygantry_value, test_object.ygantry.egu))

# X and y checks

In [ ]:
verify_axes_coupled(test_object)

In [ ]:
get_gantry_difference(test_object)

In [ ]:
# get 10 second average noise level for x and y position
avg_noise_level(test_object.xgantry)
avg_noise_level(test_object.ygantry)

Request a move, 0.5mm
Record: final settled value of primary axis, final gantry difference, time for move to complete

In [ ]:
# running the test with 5 mins timeout
gantry_checks(test_object.xgantry, 0.5, initial_xgantry_value)

In [ ]:
gantry_checks(test_object.ygantry, 0.5, initial_ygantry_value)

# Pitch checks

Record 10 second average noise level

In [ ]:
avg_noise_level(test_object.pitch)

Record time for small move (ie. move within piezo range [~20 nrad])

In [ ]:
rec_time_for_a_move(test_object, 20/1000)

Record time for large move (ie. move > 60 nrad)

In [ ]:
rec_time_for_a_move(test_object, 100/1000)

# for all axes

Request a large move, change request before move is complete (small delay), verify target change is accepted and final position matches second command


xgantry

In [ ]:
low_lim, hi_lim = low_high_limit(test_object.xgantry)
request_changeRequest(test_object.xgantry, initial_xgantry_value, hi_lim - 1, hi_lim - 0.4)

ygantry

In [ ]:
low_lim, hi_lim = low_high_limit(test_object.ygantry)
request_changeRequest(test_object.ygantry, initial_ygantry_value, hi_lim - 1, hi_lim - 0.4)

pitch

In [ ]:
low_lim, hi_lim = low_high_limit(test_object.pitch.position)
request_changeRequest(test_object.pitch, initial_mirror_angle, hi_lim -1, hi_lim - 0.4 )

In [ ]:
# Restore all axes to original position
logger.info("restoring x value to original position...")
test_object.xgantry.move(initial_xgantry_value)
logger.info("restoring y value to original position...")
test_object.ygantry.move(initial_ygantry_value)
logger.info("restoring pitch to original position...")
test_object.pitch.move(initial_mirror_angle)
logger.info("done")